In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[8]:


# Import Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import html5lib
import lxml
from splinter import Browser
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, render_template


        # In[18]:


        # Set the executable path and initialize Splinter

executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

In [2]:
def scrape():
    
    # In[13]:

    mars_info_dictionary = dict()
    # NASA Mars News 
    # Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. 
    # Assign the text to variables that you can reference later.

    # Visit the Maars News Site provided in the insructions

    url = 'https://redplanetscience.com/'
    browser.visit(url)


    # In[15]:


    # News Title

    # Using Beautiful Soup, the broswer html will be converted to a soup object.Convert the browser html to a soup object.
    # To collect the latest News Title and Paragraph Text. I am going to the site and inspect the onject to get the classes for
    # the title and paragraph. According to inspect process <div class="content_title> is the class
    # Note: Can use soup because that is the name assigned when importing in bs.

    html = browser.html
    soup = bs(html, 'html.parser')
    news_title = soup.find('div', class_='content_title')
    #news_title.text


    # In[16]:


    # After locating the headline, referenced the news_title variable and the .text.strip() functions to make the news title
    # regular text without the html markup
    #https://python.hotexamples.com/examples/bs4/BeautifulSoup/strip/python-beautifulsoup-strip-method-examples.html

    news_title = news_title.text.strip()
    #print(news_title)
    mars_info_dictionary['news_titles'] = news_title

    # In[17]:


    #News Paragraph 

    # According to the inspect process <div class="article_teaser_body">

    news_paragraph = soup.find('div', class_='article_teaser_body')
    news_paragraph

    # In[18]:


    # After locating the paragraph, referenced the news_paragraph variable and the .text.strip() functions to make the 
    # paragraph regular text without the html markup
    #https://python.hotexamples.com/examples/bs4/BeautifulSoup/strip/python-beautifulsoup-strip-method-examples.html

    news_paragraph = news_paragraph.text.strip()
    #print(news_paragraph)
    mars_info_dictionary['news_paragraph'] = news_paragraph

    # In[8]:


    # JPL Mars Space Images - Featured Image


    # In[27]:


    # Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
    # Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the 
    # url string to a variable called `featured_image_url`.
    # Make sure to find the image url to the full size `.jpg` image.
    # Make sure to save a complete url string for this image.

    url_2 = "https://spaceimages-mars.com/"
    browser.visit(url_2)


    html = browser.html
    soup = bs(html, 'html.parser')
    img = soup.find('img', class_='headerimage fade-in')
    img_link = img['src']

    featured_image_url = url_2 + img_link

    mars_info_dictionary ['featured_image'] = featured_image_url 

    # In[11]:


    ###Scrape the table containing facts about diameter of Mars
    #!pip install html5lib

    url_3 = "https://galaxyfacts-mars.com/"


    # In[13]:

    df = pd.read_html(url_3)
    #display(df[1])
    mars_df = (df[1])
    
    mars_df.columns = ['name', 'measure']
    #mars_df

    # In[ ]:

    # Converting df to html in case it is needed for the flask portion of the exercise
    #mars_df.to_html(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, sparsify=None,index_names=True, justify=None, bold_rows=True, classes=None, escape=True, max_rows=None, max_cols=None, show_dimensions=False, notebook=False)

    mars_info_dictionary['mars_facts'] = mars_df.to_dict('records')

    # In[50]:


    #Create empty dictionary for results
    hemisphere_image_urls = {}

    #Define the url
    url_4 = "https://marshemispheres.com/"

    #Go to main website
    browser.visit(url_4)

    #Read page as HTML
    html = browser.html

    #Parse HTML
    soup = bs(html, 'html.parser')

    #Find area that represents the list with hemispheres
    nav_list = soup.find('div', class_="collapsible results")

    #From the above area find every item in the list
    all_hemispheres = nav_list.find_all('div', class_="item")

    #Extract image link for each hemisphere
    for hemisphere in all_hemispheres:
        #Find hemisphere name
        hemisphere_name = hemisphere.find('h3').text

        #Find the link to hemiphere page
        link = hemisphere.find('a', class_ = "itemLink product-item")['href']

        #Create a full URL for hemishphere page
        hemisphere_link = url_4 + link

        #Go to hemisphere page
        browser.visit(hemisphere_link)

        #Read page as HTML
        html = browser.html

        #Parse HTML
        soup = bs(html, 'html.parser')

        #Extract link to full image
        img = soup.find('img', class_='wide-image')
        hemisphere_img_link = img['src']

        #Create a full URL for hemisphere image
        hemisphere_image_url = url_4 + hemisphere_img_link

        #Add hemisphere name and hemisphere url to dictionary with results
        hemisphere_image_urls[hemisphere_name] = hemisphere_image_url

    browser.quit()    
    
    #Create one dictionary with all the data
    mars_info_dictionary['hemisphere_images'] = hemisphere_image_urls

    return mars_info_dictionary


In [3]:
# conn = 'mongodb://localhost:27017'

# client = pymongo.MongoClient(conn)

# db = client.mars_db

# mars_info_dictionary = scrape()

#db.mars_info.update_one({}, {"$set": mars_info_dictionary})

In [4]:
conn = 'mongodb://localhost:27017'

client = pymongo.MongoClient(conn)

db = client.mars_db

data = list(db.mars_info.find())[0]

data

{'_id': ObjectId('622d0f0de211cf2e33c60d5c'),
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'hemisphere_images': {'Cerberus Hemisphere Enhanced': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'Schiaparelli Hemisphere Enhanced': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'Syrtis Major Hemisphere Enhanced': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'Valles Marineris Hemisphere Enhanced': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 'mars_facts': [{'name': 'Equatorial Diameter:', 'measure': '6,792 km'},
  {'name': 'Polar Diameter:', 'measure': '6,752 km'},
  {'name': 'Mass:', 'measure': '6.39 × 10^23 kg (0.11 Earths)'},
  {'name': 'Moons:', 'measure': '2 ( Phobos & Deimos )'},
  {'name': 'Orbit Distance:', 

In [ ]:
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, render_template
from scrape_mars import scrape


app = Flask(__name__)

@app.route('/')
def index():
    conn = 'mongodb://localhost:27017'

    client = pymongo.MongoClient(conn)

    db = client.mars_db
    
    mars = list(db.mars_info.find())[0]
    
    return render_template('index.html', mars=mars)


@app.route('/scrape')
def scrape():
    conn = 'mongodb://localhost:27017'

    client = pymongo.MongoClient(conn)

    db = client.mars_db
    
    mars_info_dictionary = scrape()
    
    db.mars_info.update_one({}, {"$set": mars_info_dictionary}, upsert=True)
    
    return redirect('/', code=302)


if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
